In [1]:
# first set up the and run the kafka server also install kafka-python. The commands can be found in 'kakfa commands.txt'
from kafka import KafkaAdminClient
from kafka.admin import NewTopic
from time import sleep
bootstrap_servers = "localhost:9092"

# Create an instance of KafkaAdminClient
admin_client = KafkaAdminClient(
    bootstrap_servers=bootstrap_servers,  # Update with your Kafka broker(s)
    client_id='my_client'
)

topic_name = '3DOF'

# in the first partition is the data sent to the clent from the presentation_server_data 
# in the second partition is the data sent to presentation_server_gps_correction from the client
# in the third partion is the corrected gps position sent from presentation_server_gps_correction for the client

num_partitions = 3 
replication_factor = 1
topic_configs = {"retention.ms": str(600000), # all messages that are older than 10 min in the topic get deleted
                 "retention.bytes": str(int(100e6)), #when the messages in the topic exide 100 MB old messages get deleted 
                 'max.message.bytes': str(int(5e6))} #max allowed size in MB of each message set to 5MB 

try:
    my_first_topic = NewTopic(name=topic_name, num_partitions=num_partitions,
                            replication_factor=replication_factor,
                            topic_configs=topic_configs)

    admin_client.create_topics(new_topics=[my_first_topic])
    print(f'created {topic_name}')
except:
    print(f'topic {topic_name} already existed')
    admin_client.delete_topics(topics=[topic_name]); sleep(0.2)


    admin_client.create_topics(new_topics=[my_first_topic])
    print(f'deleted old and created new {topic_name}')

# List all topics
topics = admin_client.list_topics()
print("Topics:", topics)






topic 3DOF already existed
deleted old and created new 3DOF
Topics: ['3DOF', '3DOF_start_producing_data', '3DOF_old', 'my_first_topic']


In [2]:
from kafka import KafkaConsumer, KafkaProducer,TopicPartition
import msgpack
import simplejpeg
import torch


def dict_to_bytes(data_dict):
    return msgpack.packb(data_dict, use_bin_type=True)


def bytes_to_dict(json_str):
    return msgpack.unpackb(json_str, raw=False)


def encode_pytorch_image(img):
    """
    Takes a pytorch tensor of shape (3,H,W) and encodes it to bytes, jpeg compression is used 
    img is float32 and values are in range (0,1)
    """
    img_numpy = img * 255.0 
    img_numpy = torch.permute(img_numpy, (1,2,0)).contiguous()
    img_numpy = img_numpy.to(torch.uint8).numpy()
    grd_img_bytes = simplejpeg.encode_jpeg(img_numpy)

    return grd_img_bytes

def decode_pytorch_image(img_bytes):
    """
    Takes an encoded pytorch images that is in the form of bytes and decodes it 
    return tensor of shape (3,H,W)
    """
    img_numpy = simplejpeg.decode_jpeg(img_bytes)
    img_torch = torch.tensor(img_numpy, dtype=torch.float32) / 255.0
    img_torch = torch.permute(img_torch, [2,0,1]).contiguous()

    return img_torch




producer = KafkaProducer(bootstrap_servers=bootstrap_servers)

topic_name_start = '3DOF_start_producing_data'
tp1 = TopicPartition(topic_name_start, 0)
consumer_start = KafkaConsumer(#topic = topic_name,
                         bootstrap_servers=bootstrap_servers,
                         auto_offset_reset='latest', #will start consuming from the last message in the topic 
                        #  consumer_timeout_ms=1 #stop consumer from waiting for messages after 1ms of not reciving any messages 
                         )
consumer_start.assign([tp1])

In [3]:
from Ford_dataset_s import SatGrdDatasetFordPresentation, train_logs, train_logs_img_inds, test_logs, test_logs_img_inds
import torch
from torch.utils.data import DataLoader
from train_ford_3DOF_s import parse_args

device = 'cpu'
args = parse_args()
args.train_log_start = 3

# cameras = ['FL', 'SL'] 
# args.image_H = 256 #256
# args.image_W = 1024#1024
# args.cameras = cameras
# args.batch_size = 1
# args.lifting = '3D' #homography

test_set = SatGrdDatasetFordPresentation(logs=test_logs[args.train_log_start:args.train_log_start+1],
                                logs_img_inds=test_logs_img_inds[args.train_log_start:args.train_log_start+1],
                                shift_range_lat=args.shift_range_lat, shift_range_lon=args.shift_range_lon,
                                rotation_range=args.rotation_range, whole=args.test_whole, 
                                H = args.image_H, W = args.image_W, cameras=args.cameras, mode='train') 
# testloader = DataLoader(test_set, batch_size=args.batch_size, shuffle=False, pin_memory=True,
#                              num_workers=2, drop_last=False)



len(test_set), args.batch_size

/mnt/c/Users/dimitris/Desktop/MyFiles/diplomatikh/pytorch-tensorflow/pytorch/Cross-View-Localization/server client with kafka
Error importing huggingface_hub.hf_api: cannot import name 'COMMON_SAFE_ASCII_CHARACTERS' from 'charset_normalizer.constant' (/home/dimitris/miniconda3/envs/condapy310/lib/python3.10/site-packages/charset_normalizer/constant.py)


(3511, 1)

In [6]:
while True:
    for message in consumer_start:
        break
    print('started sending data to client')
    for data_id in range(len(test_set)):
        data_local = test_set[data_id]
        sat_img, grd_imgs, gt_shift_u, gt_shift_v, gt_heading, grd_names, sat_img_norot_notran, s_lat, s_lon, g_lat, g_lon, yaw = [[camera.to(device).unsqueeze(0) if type(camera) == torch.Tensor else camera for camera in item] if type(item)== tuple else item.to(device).unsqueeze(0) if type(item) == torch.Tensor else item for item in data_local]
        

        image_0 = encode_pytorch_image(grd_imgs[0].cpu().squeeze(0))
        image_1 = encode_pytorch_image(grd_imgs[1].cpu().squeeze(0))

        data_send = {'image_0':image_0, 'image_1':image_1, 'data_id':data_id}
        data_send_bytes = dict_to_bytes(data_send)
        producer.send(topic_name, value=data_send_bytes, partition=0)
        
        print(f'data_id sent {data_id}')
        if data_id>160*1:break

started sending data to client
data_id sent 0
data_id sent 1
data_id sent 2
data_id sent 3
data_id sent 4
data_id sent 5
data_id sent 6
data_id sent 7
data_id sent 8
data_id sent 9
data_id sent 10
data_id sent 11
data_id sent 12
data_id sent 13
data_id sent 14
data_id sent 15
data_id sent 16
data_id sent 17
data_id sent 18
data_id sent 19
data_id sent 20
data_id sent 21
data_id sent 22
data_id sent 23
data_id sent 24
data_id sent 25
data_id sent 26
data_id sent 27
data_id sent 28
data_id sent 29
data_id sent 30
data_id sent 31
data_id sent 32
data_id sent 33
data_id sent 34
data_id sent 35
data_id sent 36
data_id sent 37
data_id sent 38
data_id sent 39
data_id sent 40
data_id sent 41
data_id sent 42
data_id sent 43
data_id sent 44
data_id sent 45
data_id sent 46
data_id sent 47
data_id sent 48
data_id sent 49
data_id sent 50
data_id sent 51
data_id sent 52
data_id sent 53
data_id sent 54
data_id sent 55
data_id sent 56
data_id sent 57
data_id sent 58
data_id sent 59
data_id sent 60
dat

KeyboardInterrupt: 

In [ ]:
# admin_client.delete_topics(topics=[topic_name])

DeleteTopicsResponse_v3(throttle_time_ms=0, topic_error_codes=[(topic='3DOF', error_code=0)])